In [45]:
import os
import sys
sys.path.append('/home/rustem/projects/mts')
import pandas as pd
import numpy as np
from pyarrow import parquet as pq
from src.lib.DBManager import DBManager
from src.base.settings import connector

In [27]:
def get_age_bucket(x: int) -> int:
    # age buckets
    b1 = pd.Interval(left=18, right=25, closed='right')
    b2 = pd.Interval(left=25, right=35, closed='right')
    b3 = pd.Interval(left=35, right=45, closed='right')
    b4 = pd.Interval(left=45, right=55, closed='right')
    b5 = pd.Interval(left=55, right=65, closed='right')
    b6 = pd.Interval(left=65, right=np.inf, closed='right')
    if x in b1:
        return 0
    elif x in b2:
        return 1
    elif x in b3:
        return 2
    elif x in b4:
        return 3
    elif x in b5:
        return 4
    elif x in b6:
        return 5
    else:
        return np.nan


In [23]:
db = DBManager(connector)

In [46]:
submit_ids = pq.read_table('/home/rustem/projects/mts/data/raw/submit_2.pqt').to_pandas()
train_ids = pq.read_table('/home/rustem/projects/mts/data/raw/public_train.pqt').to_pandas()
train_ids = train_ids.set_index('user_id')

In [24]:
train = db.read_data_as_df(
    """
    select
        user_id
    from
        train_agg
    """
)

In [28]:
train['age'] = train['user_id'].map(train_ids['age'])
train['age'] = train['age'].map(lambda x: get_age_bucket(x))
train['is_male'] = train['user_id'].map(train_ids['is_male'])

In [29]:
train

,user_id,age,is_male
0,0,1.0,0
1,0,1.0,0
2,0,1.0,0
3,0,1.0,0
4,0,1.0,0
...,...,...,...
14563003,415304,1.0,0
14563004,415309,1.0,0
14563005,415311,2.0,0
14563006,415312,2.0,1


In [33]:
train['age'].value_counts(normalize=True)

1.0    0.343178
2.0    0.276960
3.0    0.151034
0.0    0.134504
4.0    0.079130
5.0    0.015194
Name: age, dtype: float64

In [34]:
train['age'].value_counts()

1.0    4974207
2.0    4014408
3.0    2189177
0.0    1949578
4.0    1146953
5.0     220227
Name: age, dtype: int64

In [32]:
train['is_male'].value_counts(normalize=True)

1     0.508430
0     0.473488
NA    0.018082
Name: is_male, dtype: float64

In [43]:
submission = pd.read_csv('/home/rustem/projects/mts/data/interim/submission.csv')
submission.head()

,user_id,age,is_male
0,144141,6,0.649124
1,144144,3,0.513514
2,144145,4,0.695626
3,144146,2,0.689960
4,144149,3,0.680327


In [44]:
sample_submission = pd.read_csv('/home/rustem/projects/mts/data/raw/sample_submission.csv')
sample_submission.head()

,user_id,age,is_male
0,6,2,0.330467
1,11,5,0.725477
2,19,1,0.240190
3,27,2,0.536798
4,32,3,0.471325
